# Instructor Do: Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/soma/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv('news_api')

In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]

Total articles about Facebook Libra: 55


{'source': {'id': None, 'name': 'Gizmodo.com'},
 'author': 'Matt Novak',
 'title': "Facebook's Long-Stalled Digital Currency Could be Tested This Year: Report",
 'description': 'Facebook hopes to launch a trial of its long-stalled digital currency project by the end of this year, according to a new report from CNBC. The currency, first announced in 2019 as Libra and then renamed Diem after some bad publicity, will  now be pegged to t…',
 'url': 'https://gizmodo.com/facebooks-long-stalled-digital-currency-could-be-tested-1846719510',
 'urlToImage': 'https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/lqocl25kijqvxb53lkip.jpg',
 'publishedAt': '2021-04-20T10:00:00Z',
 'content': 'Facebook hopes to launch a trial of its long-stalled digital currency project by the end of this year, according to a new report from CNBC. The currency, first announced in 2019 as Libra and then ren… [+3626 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()

,date,text,compound,positive,negative,neutral
0,2021-04-20,Facebook hopes to launch a trial of its long-s...,0.4215,0.074,0.000,0.926
1,2021-04-08,"In June, 2019, Facebook unveiled plans to crea...",0.1779,0.059,0.039,0.901
2,2021-04-21,Pavlo Gonchar/SOPA Images/LightRocket via Gett...,0.0000,0.000,0.000,1.000
3,2021-04-15,Incoming Petrobras CEO Joaquim Silva e Luna ha...,0.2732,0.088,0.000,0.912
4,2021-04-14,"April\r\n14, 2021\r\n8 min read\r\nThis articl...",-0.3400,0.000,0.072,0.928


In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()

,compound,positive,negative,neutral
count,55.000000,55.000000,55.000000,55.000000
mean,0.136402,0.075200,0.032509,0.892236
std,0.438569,0.097753,0.043313,0.097638
min,-0.743000,0.000000,0.000000,0.595000
25%,-0.102350,0.000000,0.000000,0.851500
50%,0.077200,0.054000,0.000000,0.912000
75%,0.430950,0.100500,0.068000,0.946500
max,0.957800,0.405000,0.174000,1.000000
